<a href="https://colab.research.google.com/github/DoryAbelman/CARTE-ML-Program/blob/main/lab_4_2_embeddings_gene2vec_dory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gene2Vec

Gene2Vec is a method to learn gene embeddings from gene expression data. The method is based on the Word2Vec model, which is a popular method to learn word embeddings from text data. The idea is to learn a low-dimensional representation of genes that captures the relationships between genes based on their expression profiles. The gene embeddings can then be used for various downstream tasks, such as gene function prediction, gene expression analysis, and gene network analysis.

Gene2Vec was trained to produce 200-dimensional vectors for all human genes, using co-expression data obtained from 984 datasets. We can load it below:

In [1]:
gene2vec_url = 'https://raw.githubusercontent.com/jingcheng-du/Gene2vec/master/pre_trained_emb/gene2vec_dim_200_iter_9_w2v.txt'

# Loading Gene2Vec

We can load the Gene2Vec model using the Gensim library. Gensim is a popular library for working with word embeddings, and it provides an easy-to-use interface for loading and working with pre-trained word embeddings. Let's load the Gene2Vec model and explore the embeddings:

In [2]:
!pip install -U -q gensim

In [3]:
from gensim.models import KeyedVectors

gene2vec = KeyedVectors.load_word2vec_format(gene2vec_url)

Now the word embeddings are loaded inside the `gene2vec` object. We can use this object to access the embeddings for individual genes and perform various operations, such as finding similar genes or performing vector arithmetic on gene embeddings. Let's explore some of these operations:

Load the embeddings for a specific gene:

In [4]:
gene2vec['TP53']

array([-0.17541103,  0.30231935,  0.02986403, -0.23244244, -0.24435847,
        0.04649721,  0.27385196, -0.38778004,  0.05505791, -0.00124642,
       -0.16722302,  0.287758  , -0.15370846,  0.06252019,  0.02235236,
       -0.12657824,  0.2909681 , -0.03715301, -0.2229012 , -0.1496548 ,
        0.12428768, -0.13092335, -0.10339656, -0.08450292,  0.13020161,
        0.14910421, -0.02058535, -0.12705182, -0.1530847 ,  0.1781307 ,
       -0.291634  ,  0.16681159, -0.15853739,  0.17557888,  0.31498173,
        0.11993543,  0.12918676, -0.21697438, -0.27889672,  0.04069283,
       -0.4284741 ,  0.00899589,  0.20971633, -0.40938446,  0.30795485,
       -0.30104858, -0.35495016,  0.18944286,  0.05598959, -0.06362291,
        0.17820194,  0.06856135, -0.3468939 , -0.00121399, -0.04772829,
        0.07169076, -0.16076994,  0.21976957, -0.13015388, -0.36424658,
       -0.02852698, -0.37614885,  0.00780825, -0.15109184, -0.39694637,
       -0.0613759 , -0.17475432, -0.22634187,  0.05431679, -0.14

Find the most similar genes to a given gene:

In [5]:
gene2vec.most_similar('TP53')

[('TRRAP', 0.7597607970237732),
 ('STARD7', 0.6233418583869934),
 ('UVRAG', 0.6176624298095703),
 ('TMEM64', 0.6125375628471375),
 ('SLC25A11', 0.6100547313690186),
 ('TOMM70A', 0.6036673784255981),
 ('ZBTB14', 0.601607620716095),
 ('ZNF275', 0.6006855368614197),
 ('UBFD1', 0.5981127023696899),
 ('USP7', 0.5974504351615906)]

Find the most similar genes to a combination of genes - this is a bit more complex, but you can think of this operation as finding genes that are similar to the combination of the two genes and different from the third gene:

In [7]:
gene2vec.most_similar(positive=['TP53', 'BRCA1'], negative=['BRCA2'])

[('TMEM254', 0.5834333300590515),
 ('ZDHHC16', 0.5538336634635925),
 ('STARD7', 0.544048547744751),
 ('TOR3A', 0.5391254425048828),
 ('WIPF2', 0.5293912291526794),
 ('SUPT7L', 0.5262064933776855),
 ('SURF4', 0.525196373462677),
 ('RSL1D1', 0.5201980471611023),
 ('SLC25A11', 0.5193983912467957),
 ('ZNF133', 0.5136620998382568)]

We can also apply dimensionality reduction to visualize the genes. For this, we'll use the Plotly library, because it'll allow us to mouse over the points and see the gene names. Let's visualize the gene embeddings using UMAP:

In [8]:
!pip install -U -q umap-learn plotly

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 73.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 7.9 MB/s eta 0:00:00


In [9]:
import umap

embedding = umap.UMAP(n_neighbors=15, min_dist=0.1, metric='cosine').fit_transform(gene2vec.vectors)

In [10]:
import plotly.express as px

fig = px.scatter(x=embedding[:, 0], y=embedding[:, 1], text=gene2vec.index_to_key)
fig.update_traces(textposition='top center')
fig.update_layout(title='Gene2Vec Embeddings')
fig.show()

# Augmenting data using embeddings

We can take advantage of these embeddings to augment performance in an existing task. Let's see how introducing these embeddings can improve our results on the HCC data:

In [11]:
import pandas as pd
hcc = pd.read_csv('https://github.com/alexwolson/carte_workshop_2024/raw/main/data/HCC_all_ML_classification_test_annotated_frags_all_features_combined_4_tumors.csv.gz', compression='gzip')

In [12]:
vocab = set(gene2vec.index_to_key)

In [13]:
hcc = hcc[hcc.gene.isin(vocab)]

In [14]:
hcc = hcc.sample(2000)

In [25]:
categorical_columns = ['upstream_motif','downstream_motif','Corrected_Call']
numerical_columns = ['frag','VAF','plasma_VAF','Corrected_Copy_Number']
y_column = 'alt_match'

X = hcc[categorical_columns + numerical_columns]
y = hcc[y_column]

In [26]:
X = pd.get_dummies(X, columns=categorical_columns)

In [27]:
X = X.values

In [28]:
import numpy as np

gene_embeddings = hcc.gene.map(gene2vec.get_vector).values
gene_embeddings = np.stack(gene_embeddings)

print(gene_embeddings.shape)

(2000, 200)


In [29]:
X = np.concatenate([X, gene_embeddings], axis=1)

In [30]:
print(X.shape)

(2000, 688)


In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training a model

Now, train a model on the augmented data. You can import any model you'd like, such as a Random Forest or Gradient Boosting Machine.

In [32]:
## Random forest
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(max_depth=5, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

## Get scores
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

# Evaluate the model on the test set
score = model.score(X_test, y_test)
print(f"Test score: {score}")


Accuracy: 0.97
Precision: 0.0
Recall: 0.0
F1 score: 0.0
Test score: 0.97


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning:

Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.



In [33]:
gene_embeddings

array([[ 0.08580936,  0.03944207,  0.10086193, ..., -0.46570882,
        -0.07417711, -0.28027347],
       [-0.5712848 , -0.23517647,  0.29637468, ...,  0.13006546,
        -0.11679693, -0.464662  ],
       [ 0.15831363,  0.09031276, -0.46313888, ..., -0.09151342,
        -0.44918028,  0.19846727],
       ...,
       [-0.1873234 ,  0.0366945 , -0.1747898 , ...,  0.05000604,
        -0.1461106 , -0.15417895],
       [ 0.08179974,  0.00813217,  0.47665197, ...,  0.08040453,
        -0.00129134,  0.01388578],
       [-0.29137546, -0.41942286,  0.00522673, ...,  0.1746983 ,
         0.30204543, -0.87485176]], dtype=float32)